In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.io import wavfile
import os
from pydub import AudioSegment
from matplotlib.pyplot import *

%matplotlib inline

In [2]:
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('talk')

In [6]:
df = pd.read_feather('vocalization_df.feather')

df_c2 = df[df.cohort=='c2']
df_c4 = df[df.cohort=='c4']
df_c5 = df[df.cohort=='c5']

z = df.groupby('z_70')['timestamp'].count().values
reorder = np.argsort(z)[::-1]

In [7]:
states_reorder = np.array([0,3,10,15,21,23,25,
                           42,45,54,56,64,67])
states = [np.where(reorder == states_reorder[i])[0][0] for i in range(len(states_reorder))]

In [ ]:
n_samples = 100
fs = 125000
audio_basename = 'cohort2_combined_audio'

for state in states:
    print('Working on state {}'.format(state))
    
    #take longest duration examples for each state
    df_sub = df_c2[df_c2.z_70 == state]
    longest_dur_sort = np.argsort(df_sub.offset-df_sub.onset)[::-1].values
    df_sub = df_sub.iloc[longest_dur_sort][:n_samples]
    
    #plot
    figure(figsize=(10,10))
    for i in range(n_samples):
        working_onset, working_offset = df_sub.onset.values[i], df_sub.offset.values[i]
        working_filename = df_sub.audio_filename.values[i]
        
        working_audio_slice = AudioSegment.from_file(os.path.join(audio_basename, working_filename), 
                                             codec='pcm_f64le', 
                                             start_second=working_onset, 
                                             duration=working_offset-working_onset)

        working_audio = np.array(working_audio_slice.get_array_of_samples())
    
        v = np.zeros(int(.3*fs))
        v[int((len(v)/2) - len(working_audio)/2):int((len(v)/2) +len(working_audio)/2)] = working_audio

        subplot(10,10,i+1)
        specgram(v, NFFT=512, noverlap=256, Fs=fs,clim=(-10,20), cmap='magma')
        xticks([])
        yticks([])
        #axis('equal')
    
    savefig('multi_syllababic_examples_state_{}.png'.format(state), dpi=300)
    close()